# LIONS Analysis

Our project is to analyze federal cases.

In [1]:
from lions.parse import *
import pandas as pd
import numpy as np

In [2]:

case_sql, cases = gs_case()
hist_sql, hist = gs_court_hist()
court_judge_sql, court_judge = gs_court_judge()
judge_sql, judge = gs_judge()



In [4]:
cases = cases[:5000]
hist = hist[:5000]
court_judge = court_judge[:5000]

cases_df = pd.DataFrame(cases, columns = cases[0].keys())
hist_df = pd.DataFrame(hist, columns = hist[0].keys())
court_judge_df = pd.DataFrame(court_judge, columns = court_judge[0].keys())
judge_df = pd.DataFrame(judge, columns = judge[0].keys())


print(len(cases_df))
print(len(hist_df))
court_judge_df



5000
5000


,DISTRICT,CASEID,CRTHISID,ID,JUDGEID,DECISION,START_DATE,END_DATE,CREATE_DATE,CREATE_USER,UPDATE_DATE,UPDATE_USER
0,AK,1983V01272,1,1366,6,,06-JUL-1983,,,,,
1,AK,1986V01040,1,1,6,,13-FEB-1986,18-OCT-1994,,,,
2,AK,1988R00082,2757,1575,11,,10-OCT-1989,,,,,
3,AK,1989R00022,2961,1682,6,,24-JUN-2000,,,,,
4,AK,1989R00022,2821,1617,6,,23-MAY-2000,,,,,
5,AK,1989R00022,2301,1261,9,,02-MAR-1995,,,,,
6,AK,1989R00022,13207,5279,6,,30-MAR-2006,,,,,
7,AK,1989R00036,1,1,11,,21-OCT-1980,17-OCT-1995,,,,
8,AK,1989R00146,1,1,9,,24-JUN-1992,27-OCT-1995,,,,
9,AK,1989V00145,1,1,6,,07-JUL-1989,13-JUN-1996,,,,


In [43]:



cases_hist_df = pd.merge(cases_df, hist_df, left_on = 'ID', right_on = 'CASEID', how = 'inner')

cases_hist_court_df = pd.merge(cases_hist_df, court_judge_df, left_on = 'CASEID', right_on = 'CASEID', how = 'inner')

cases_hist_court_judge_df = pd.merge(cases_hist_court_df, judge_df, left_on = ['JUDGEID','DISTRICT'], right_on = ['ID','DISTRICT'], how = 'inner')

print(len(cases_hist_df))
print(len(cases_hist_court_df))
print(len(cases_hist_court_judge_df))


withdisp_df = cases_hist_court_judge_df[cases_hist_court_judge_df.DISPOSITION != '']
print(len(withdisp_df))
withdisp_reason_df = withdisp_df[withdisp_df.DISP_REASON1 != '']
print(len(withdisp_reason_df))

print(withdisp_reason_df.columns)


5000
5377
5377
5369
4152
Index(['DISTRICT_x', 'ID_x', 'CLASS', 'NAME', 'RECVD_DATE', 'SECURITY',
       'PRIORITY', 'TYPE_x', 'US_ROLE_x', 'LIT_RESP', 'LIT_TRACK', 'WEIGHT',
       'BRANCH', 'GRAND_JURY_NUM', 'UNIT', 'DCMNS_NUMBER', 'TRIBE',
       'RESERVATION', 'SPEC_PROJ', 'VIC_WIT', 'ADR_MODE', 'COLLECT_IND',
       'OFFENSE_FROM', 'OFFENSE_TO', 'LEAD_CHARGE', 'PHYSICAL_LOC',
       'STORE_NUM', 'CIVIL_POTEN', 'SYS_INIT_DATE', 'ACCESS_DATE', 'STATUS',
       'CLOSE_DATE', 'DEST_DATE', 'PERMANENT', 'CASE_RESTRICTED',
       'CRIMINAL_POTEN', 'TOT_VICTIMS', 'RELATED_FLU_FLAG', 'QUI_TAM',
       'CREATE_DATE_x', 'CREATE_USER_x', 'UPDATE_DATE_x', 'UPDATE_USER_x',
       'DISTRICT_y', 'CASEID', 'ID_y', 'COURT', 'LOCATION', 'US_ROLE_y',
       'COURT_NUMBER', 'FILING_DATE', 'SERVICE_DATE', 'TRIAL_DAYS',
       'NOAP_DATE', 'APPEAL_TYPE', 'SENT_APPEAL', 'DISPOSITION', 'DISP_DATE',
       'DISP_REASON1', 'DISP_REASON2', 'DISP_REASON3', 'SYS_DISP_DATE',
       'SYS_FILING_DATE', 'CREATE_DAT

In [47]:
columns_to_drop = ['ID_x','NAME','RECVD_DATE','SECURITY','PRIORITY', 'TYPE_x', 'US_ROLE_x','LIT_RESP', \
                   'LIT_TRACK','WEIGHT','BRANCH','GRAND_JURY_NUM', 'UNIT', 'DCMNS_NUMBER', 'TRIBE','RESERVATION', \
                   'SPEC_PROJ', 'VIC_WIT', 'ADR_MODE', 'COLLECT_IND','OFFENSE_FROM', 'OFFENSE_TO', 'PHYSICAL_LOC', \
                   'STORE_NUM','SYS_INIT_DATE', 'ACCESS_DATE', 'STATUS','CLOSE_DATE', 'DEST_DATE', 'PERMANENT', \
                   'RELATED_FLU_FLAG', 'QUI_TAM','CREATE_DATE_x', 'CREATE_USER_x', 'UPDATE_DATE_x', 'UPDATE_USER_x', \
                   'DISTRICT_y', 'CASEID', 'ID_y', 'LOCATION', 'US_ROLE_y','COURT_NUMBER', 'FILING_DATE', 'SERVICE_DATE', \
                   'NOAP_DATE','DISP_DATE','DISP_REASON1', 'DISP_REASON2', 'DISP_REASON3', 'SYS_DISP_DATE','SYS_FILING_DATE', \
                   'CREATE_DATE_y', 'CREATE_USER_y', 'UPDATE_DATE_y','UPDATE_USER_y', 'DISTRICT', 'CRTHISID', 'ID_x', 'JUDGEID', \
                   'START_DATE', 'END_DATE', 'CREATE_DATE_x', 'CREATE_USER_x','UPDATE_DATE_x', 'UPDATE_USER_x', 'ID_y', 'INITIALS', \
                   'INIT_STAT','LAST_NAME', 'FIRST_NAME', 'COURT_ROOM', 'TYPE_y', 'CREATE_DATE_y','CREATE_USER_y', 'UPDATE_DATE_y','UPDATE_USER_y', \
                   'CIVIL_POTEN','CASE_RESTRICTED','CRIMINAL_POTEN','TOT_VICTIMS']

disp_reasons = ['VACA','VARM','RDAP','RDRR','REVA','REVR','OFPO','GWDA','DEPO','LECI','WKEV','WTPR']


misconduct = []
disp1 = withdisp_reason_df['DISP_REASON1'].tolist()

for i in range(len(disp1)):
    if disp1[i] in disp_reasons: 
        misconduct.append(1)
    else:
        misconduct.append(0)

df_classify = withdisp_reason_df.drop(columns_to_drop, axis = 1)
df_classify

,DISTRICT_x,CLASS,LEAD_CHARGE,COURT,TRIAL_DAYS,APPEAL_TYPE,SENT_APPEAL,DISPOSITION,DECISION
2,AK,R,18 :01005,CA,,A,,FA,
3,AK,R,18 :01005,CA,,A,,FA,
4,AK,R,18 :01005,CA,,A,,FA,
5,AK,R,18 :01005,PD,,,,DM,
6,AK,R,18 :01005,PD,,,,DM,
7,AK,R,18 :01005,PD,,,,DM,
8,AK,R,18 :01005,CA,,A,,FA,
9,AK,R,18 :01005,CA,,A,,FA,
10,AK,R,18 :01005,CA,,A,,FA,
11,AK,R,18 :01005,CA,,A,,FA,
